Example taken from https://www.kaggle.com/fatmakursun/pyspark-ml-tutorial-for-beginners

Dataset from https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

In [ ]:
!wget https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.tgz -P data/cal_housing/

In [ ]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

from matplotlib import rcParams
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})
rcParams['figure.figsize'] = 18,4

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# setting random seed for notebook reproducability
rnd_seed=23
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

In [ ]:
# define the schema, corresponding to a line in the csv data file.
schema = StructType([
    StructField("long", FloatType(), nullable=True),
    StructField("lat", FloatType(), nullable=True),
    StructField("medage", FloatType(), nullable=True),
    StructField("totrooms", FloatType(), nullable=True),
    StructField("totbdrms", FloatType(), nullable=True),
    StructField("pop", FloatType(), nullable=True),
    StructField("houshlds", FloatType(), nullable=True),
    StructField("medinc", FloatType(), nullable=True),
    StructField("medhv", FloatType(), nullable=True)]
)

In [ ]:
# Load housing data
housing_path='data/cal_housing/CaliforniaHousing/cal_housing.data'
housing_df = spark.read.csv(path=housing_path, schema=schema).cache()

# Exploratory Analysis

In [ ]:
housing_df.show()

In [ ]:
housing_df.summary().toPandas()

In [ ]:
(housing_df.describe().select(
                    "summary",
                    F.round("medage", 4).alias("medage"),
                    F.round("totrooms", 4).alias("totrooms"),
                    F.round("totbdrms", 4).alias("totbdrms"),
                    F.round("pop", 4).alias("pop"),
                    F.round("houshlds", 4).alias("houshlds"),
                    F.round("medinc", 4).alias("medinc"),
                    F.round("medhv", 4).alias("medhv"))
                    .show())

In [ ]:
# group by housingmedianage and see the distribution
result_df = housing_df.groupBy("medage").count().sort("medage", ascending=False)
result_df.show(10)

In [ ]:
result_df.toPandas().plot.bar(x='medage',figsize=(14, 6))

# Data preparation

In [ ]:
# Adjust the values of `medianHouseValue`
reformated_df = housing_df.withColumn("medhv", col("medhv")/100000)

In [ ]:
reformated_df.show(2)

In [ ]:
# Add the new columns to `df`
reformated_df = (reformated_df.withColumn("rmsperhh", F.round(col("totrooms")/col("houshlds"), 2))
                       .withColumn("popperhh", F.round(col("pop")/col("houshlds"), 2))
                       .withColumn("bdrmsperrm", F.round(col("totbdrms")/col("totrooms"), 2)))

In [ ]:
# Inspect the result
reformated_df.show(5)

In [ ]:
# Re-order and select columns
reformated_df = reformated_df.select("medhv", 
                              "totbdrms", 
                              "pop", 
                              "houshlds", 
                              "medinc", 
                              "rmsperhh", 
                              "popperhh", 
                              "bdrmsperrm")

In [ ]:
reformated_df.show(5)

In [ ]:
featureCols = ["totbdrms", "pop", "houshlds", "medinc", "rmsperhh", "popperhh", "bdrmsperrm"]

In [ ]:
# put features into a feature vector column
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

In [ ]:
assembled_df = assembler.transform(reformated_df)

In [ ]:
assembled_df.show(10, truncate=False)

In [ ]:
# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

In [ ]:
# Fit the DataFrame to the scaler
scaled_df = standardScaler.fit(assembled_df).transform(assembled_df)

In [ ]:
# Inspect the result
scaled_df.select("features", "features_scaled").show(10, truncate=False)

# Build and train a model

In [ ]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.8,.2], seed=rnd_seed)

In [ ]:
# Initialize `lr`
lr = (LinearRegression(featuresCol='features_scaled', labelCol="medhv", predictionCol='predmedhv', 
                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))

In [ ]:
# Fit the data to the model
linearModel = lr.fit(train_data)

In [ ]:
# Coefficients for the model
linearModel.coefficients

In [ ]:
# Intercept for the model
linearModel.intercept

# Predictions

In [ ]:
# Generate predictions
predictions = linearModel.transform(test_data)

In [ ]:
# Extract the predictions and the "known" correct labels
predandlabels = predictions.select("predmedhv", "medhv")

In [ ]:
predandlabels.show()

In [ ]:
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='rmse')
print("RMSE: {0}".format(evaluator.evaluate(predandlabels)))

In [ ]:
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='r2')
print("R2: {0}".format(evaluator.evaluate(predandlabels)))

# Build Pipeline

In [ ]:
from pyspark.ml.feature import SQLTransformer
from pyspark.ml import Pipeline, PipelineModel

In [ ]:
sqlTrans = SQLTransformer(statement="""
SELECT
medhv/100000 as medhv,
totbdrms, pop, houshlds, medinc, 
round(totrooms/houshlds,2) as rmsperhh,
round(pop/houshlds,2) as popperhh,
round(totbdrms/totrooms,2) as bdrmsperrm
FROM __THIS__"""
)

In [ ]:
reformated_df = sqlTrans.transform(housing_df)
reformated_df.show(10)

In [ ]:
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

In [ ]:
assembled_df = assembler.transform(reformated_df)
assembled_df.show(10, truncate=False)

In [ ]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

In [ ]:
scaler_model = standardScaler.fit(assembled_df)
scaled_df = scaler_model.transform(assembled_df)
scaled_df.select('features', 'features_scaled').show(10, truncate=False)

In [ ]:
#transform_pipeline=PipelineModel(stages=[sqlTrans, assembler])
transform_pipeline=PipelineModel(stages=[sqlTrans, assembler, scaler_model])
#transform_pipeline = pipeline.transform(housing_df)
transform_pipeline

In [ ]:
transform_pipeline.transform(housing_df).select('features', 'features_scaled').show(10, truncate=False)

In [ ]:
#pipeline = Pipeline(stages=[sqlTrans, assembler, standardScaler, lr])
pipeline = Pipeline(stages=[sqlTrans, assembler, scaler_model, lr])

In [ ]:
train_data, test_data = housing_df.randomSplit([.8,.2], seed=rnd_seed)

In [ ]:
linModel = pipeline.fit(train_data)

In [ ]:
predictions = linModel.transform(test_data)
predictions.select("predmedhv", "medhv").show(10)

In [ ]:
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='rmse')
print("RMSE: {0}".format(evaluator.evaluate(predictions)))

In [ ]:
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='r2')
print("R2: {0}".format(evaluator.evaluate(predictions)))